In [2]:
import json
import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy
import numpy as np
#import plotly.express as px
from sklearn.metrics.pairwise import cosine_similarity
pd.set_option('display.max_columns', None)

import time

In [3]:
df = pd.read_csv('../Data clean/data_clusters.csv', sep = ',', usecols = ['Codigo_Cliente','Fecha_compra','Factura','Ciudad Tienda','Desc. Tienda','Tipo de Artículo','ciudad','cluster'])

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
df.head(2)

In [ ]:
#df_virtual = df[df['Desc. Tienda'].str.contains('VIRTUAL')]

In [4]:
df_final = df[~df['Desc. Tienda'].str.contains('VIRTUAL')]
del df
#del df_virtual

In [5]:
colk=["ciudad","Codigo_Cliente","cluster"]
dfk = pd.read_csv('../Data clean/clusters.csv', sep = ',', usecols=colk)
dfk.head(2)

,Codigo_Cliente,ciudad,cluster
0,102510,AGUACHICA,100
1,115566,AGUACHICA,103


**User Item Matrix**

In [ ]:
def user_item_mat(df, city, k):    
    df_city = df[df['Ciudad Tienda'] == city]
    #k = 600
    df_k = df_final[df_final['cluster']==k]
    df_cli_art = df_k.groupby(['Codigo_Cliente','Tipo de Artículo'])['Factura'].count().reset_index().rename(columns = {'Factura':'Cantidad'})
    df_cli_art['Comprado'] = df_cli_art['Cantidad']/df_cli_art['Cantidad']
    df_cli_art['Comprado'] = df_cli_art['Comprado'].astype(int)
    #customer_item_matrix = df_cli_art.pivot_table(index='Codigo_Cliente',
    #                                          columns='Tipo de Artículo',
    #                                          values='Comprado')
    #                                          #aggfunc='sum')
    #customer_item_matrix = customer_item_matrix.fillna(0)
    df_cli_art = df_cli_art.fillna(0)
    df_cli_art['ciudad'] = city
    df_cli_art['cluster'] = k
    return df_cli_art[['Codigo_Cliente','Tipo de Artículo','Comprado','ciudad','cluster']]

In [ ]:
%%time
#df_clusters = df_data_clusters[~df_data_clusters['Ciudad Tienda'].isin(['MEDELLIN','BOGOTÁ','LA CEJA'])]
#ciudades_tiendas = df_clusters['Ciudad Tienda'].unique()
ciudades_tiendas = dfk.ciudad.unique()
#ciudades_tiendas = ['CARTAGENA']
df_cli_art = pd.DataFrame(columns = ['Codigo_Cliente', 'Tipo de Artículo', 'Comprado', 'ciudad', 'cluster']) 
for city in ciudades_tiendas:
    df_city = df_final[df_final['Ciudad Tienda'] == city]
    clusters = df_city['cluster'].unique()
    print(city, df_city.shape)
    for k in clusters:
        if df_city.shape[0] > 0:
            df_cli_art = df_cli_art.append(user_item_mat(df_city, city, k))
        else:
            print(city, k)

In [ ]:
df_cli_art.to_csv('cliente_articulo.csv')

**Cosine Similarity**

In [6]:
def sim_user_user_city(df, city, k):
    df_city = df[df['Ciudad Tienda'] == city]
    #k = 600
    df_k = df_final[df_final['cluster']==k]
    df_cli_art = df_k.groupby(['Codigo_Cliente','Tipo de Artículo'])['Factura'].count().reset_index().rename(columns = {'Factura':'Cantidad'})
    df_cli_art['Comprado'] = df_cli_art['Cantidad']/df_cli_art['Cantidad']
    df_cli_art['Comprado'] = df_cli_art['Comprado'].astype(int)
    customer_item_matrix = df_cli_art.pivot_table(index='Codigo_Cliente',
                                              columns='Tipo de Artículo',
                                              values='Comprado')
                                              #aggfunc='sum')
    customer_item_matrix = customer_item_matrix.fillna(0)
    
    user_user_sim_matrix = pd.DataFrame(cosine_similarity(customer_item_matrix))
    user_user_sim_matrix.columns = customer_item_matrix.index
    user_user_sim_matrix['Codigo_Cliente'] = customer_item_matrix.index
    user_user_sim_matrix = user_user_sim_matrix.set_index('Codigo_Cliente')
    user_user_sim_matrix.index.name = 'codigo_cliente_1'
    user_user_sim_matrix.columns.name = 'codigo_cliente_2'

    user_user_unpivot = user_user_sim_matrix.unstack().reset_index(name='sim_value')
    user_user_unpivot['ciudad'] = city
    user_user_unpivot['cluster'] = k
    return(user_user_unpivot)


In [7]:
%%time
#df_clusters = df_data_clusters[~df_data_clusters['Ciudad Tienda'].isin(['MEDELLIN','BOGOTÁ','LA CEJA'])]
#ciudades_tiendas = df_clusters['Ciudad Tienda'].unique()
#ciudades_tiendas = dfk.ciudad.unique()#['BARRANQUILLA','VALLEDUPAR']
#ciudades_tiendas = ['VALLEDUPAR']
ciudades_tiendas = ['BARRANQUILLA']
recoms_user_user_ciudad = pd.DataFrame(columns = ['codigo_cliente_2', 'codigo_cliente_1', 'sim_value', 'ciudad', 'cluster']) 
for city in ciudades_tiendas:
    df_city = df_final[df_final['Ciudad Tienda'] == city]
    clusters = df_city['cluster'].unique()
    print(city, df_city.shape)
    for k in clusters:
        if df_city.shape[0] > 0:
            recoms_user_user_ciudad = recoms_user_user_ciudad.append(sim_user_user_city(df_city, city, k))
        else:
            print(city, k)

BARRANQUILLA (73896, 8)
CPU times: user 37.2 s, sys: 32.4 s, total: 1min 9s
Wall time: 1min 7s


In [9]:
recoms_user_user_ciudad.shape

(88623419, 5)

In [11]:
import sys
sys.getsizeof(recoms_user_user_ciudad)

14622864159

In [14]:
14622864159/(1024**2)

13945.449980735779

In [ ]:
recoms_user_user_ciudad.cluster.unique()

In [ ]:
df_final[df_final['Ciudad Tienda']=='VALLEDUPAR'].cluster.unique()

In [ ]:
%%time
#df_clusters = df_data_clusters[~df_data_clusters['Ciudad Tienda'].isin(['MEDELLIN','BOGOTÁ','LA CEJA'])]
#ciudades_tiendas = df_clusters['Ciudad Tienda'].unique()
ciudades_tiendas = dfk.ciudad.unique()#['BARRANQUILLA','VALLEDUPAR']
recoms_art_art_ciudad = pd.DataFrame(columns = ['tipo_articulo_2', 'tipo_articulo_1', 'sim_value', 'ciudad', 'cluster']) 
for city in ciudades_tiendas:
    df_city = df_data_clusters[df_data_clusters['Ciudad Tienda'] == city]
    clusters = df_city['cluster'].unique()
    print(city, df_city.shape)
    for k in clusters:
        if df_city.shape[0] > 0:
            recoms_art_art_ciudad = recoms_art_art_ciudad.append(sim_item_item_city(df_city, city, k))
        else:
            print(city, k)

In [ ]:
def user_based_recoms (user, n_recoms, variab):
    #user = 102514
    most_sim_user = user_user_sim_matrix.loc[user].sort_values(ascending=False).reset_index().Codigo_Cliente[1:51]

    items_bought_by_A = pd.DataFrame(customer_item_matrix.loc[user].iloc[customer_item_matrix.loc[user].nonzero()].index)

    items_bought_by_sim_users = pd.DataFrame(customer_item_matrix.loc[most_sim_user[1]].iloc[customer_item_matrix.loc[most_sim_user[1]].nonzero()].index)
    for us in most_sim_user[1:]:
        items_bought_by_sim_users = items_bought_by_sim_users.merge(pd.DataFrame(customer_item_matrix.loc[us].iloc[customer_item_matrix.loc[us].nonzero()].index), how = 'outer')

    items_to_recommend_to_A = items_bought_by_sim_users[~items_bought_by_sim_users['Tipo de Artículo']\
                                                        .isin(items_bought_by_A['Tipo de Artículo'])]\
                                .reset_index(drop = True)[0:n_recoms]
    print('Recomendados para el cliente',user,'en',variab)
    return(items_to_recommend_to_A)
    